In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
engine = sqlite3.connect('./src/data.db')
catalyst = pd.read_sql('SELECT * FROM catalysts', engine)

In [3]:
catalyst

,ticker,disease,stage,date,catalyst
0,SNSE,Various cancers,Phase 1,2023-11-04 00:00:00,Phase 1 safety data reported that a total of 1...
1,REGN,Various cancers,Phase 1,2023-11-04 00:00:00,Phase 1 safety data reported that a total of 1...
2,AUPH,Lupus Nephritis,Phase 3,2023-11-03 00:00:00,Phase 3 data from ASN reported that treated pa...
3,IPHA,Mycosis Fungoides,Phase 2,2023-11-03 00:00:00,Phase 2 data from ASH abstract reported that t...
4,HOWL,Solid Tumors,Phase 1b,2023-11-03 00:00:00,Phase 1/1b preliminary data provided compellin...
...,...,...,...,...,...
3910,SPPI,Colorectal cancer,Approved,2011-04-29 00:00:00,"Approved April 29, 2011."
3911,ASRT,Colorectal cancer,Approved,2011-04-29 00:00:00,"Approved April 29, 2011."
3912,ASRT,Postherpetic neuralgia - shingles,Approved,2011-01-28 00:00:00,"Approved January 28, 2011."
3913,JAZZ,Fibromyalgia,CRL,2010-10-11 00:00:00,"CRL received October 11, 2010."
